# 📘 Example: Database Builder

In this notebook, we demonstrate how to use the **Database builder** API of FloodAdapt, which greatly simplifies the process of setting up a FloodAdapt database in a new location!

The most critical components of a FloodAdapt database are the [**SFINCS**](https://github.com/Deltares/SFINCS) and [**Delft-FIAT**](https://github.com/Deltares/Delft-FIAT) models, both of which can be generated with great ease using the [**HydroMT-SFINCS**](https://deltares.github.io/hydromt_sfincs/latest/) and the [**HydroMT-FIAT**](https://deltares.github.io/hydromt_fiat/stable/) plugins of [**HydroMT**](https://deltares.github.io/hydromt/latest/).

For this notebook, we will use an example area in Charleston, USA, for which we have already generated a SFINCS and a Delft-FIAT model.

In order to use the **DatabaseBuilder** of FloodAdapt a set of **configuration** parameters are needed. The **configuration** parameters can be divided to **mandatory** and **optional** ones. Using only the mandatory parameters (i.e., baseline FloodAdapt configuration) will result in a simple but functional version of FloodAdapt. By adding optional parameters to your configuration, you can create a more advanced FloodAdapt database with additional features. 

If you want to learn more about the configuration parameters, please refer to the [Database-Builder](../../../4_system_setup/database.qmd) of the Setup Guide in the documentation.

The configuration can be either created through available FloodAdapt classes or can be parsed as a simple dictionary. We advice you to work with the FloodAdapt classes, since this can avoid using wrong parameter names or values with the help of type hinting. 

In this notebook we will start by creating a simple FloodAdapt database using the baseline FloodAdapt configuration ([Step-2: Build a basic FloodAdapt database](#-step-2-build-a-basic-floodadapt-database)), and then we will go through all the optional configuration parameters and create a more complex database ([Step-3: Build an advanced FloodAdapt Database](#️-step-3-build-an-advanced-floodadapt-model)).

## Import libraries

In [ ]:
# Import packages
import pandas as pd
import geopandas as gpd
import toml
from pathlib import Path
from hydromt_fiat.fiat import FiatModel
from hydromt_sfincs.sfincs import SfincsModel
import flood_adapt.database_builder as db
from flood_adapt import FloodAdapt
from flood_adapt import Settings
from flood_adapt import unit_system as us
from flood_adapt.config.sfincs import FloodModel
from flood_adapt.objects.forcing.tide_gauge import TideGaugeSource
from flood_adapt.config.sfincs import ObsPointModel
from flood_adapt.config.sfincs import SlrScenariosModel
%matplotlib inline

## 🔍 **Step 1**. Explore the SFINCS and Delft-FIAT model

The study area is in **Charleston, USA**, a coastal city on the East Coast of the United States. To run this notebook, we have already prepared a SFINCS model and a Delft-FIAT model for this area. Both these models are simplified and are used for demonstration purposes only.

We can first inspect the extents of our SFINCS model, by loading the model with the HydroMT-SFINCS plugin.

In [ ]:
# Define the static data folder
STATIC_DATA_DIR = Path("../../../_data/examples/static-data/1_DatabaseBuilder").resolve()
# Get the path of the SFINCS overland model
fn_sfincs = STATIC_DATA_DIR  / "overland"
# Use HydroMT-SFINCS to read the SFINCS model
sfincs = SfincsModel(root=str(fn_sfincs), mode="r")
sfincs.read()
# Get the extent of the SFINCS model
gdf = sfincs.region[["geometry"]]
gdf["name"] = "SFINCS Model Extent"
# Make a map of the SFINCS model extent
gdf.explore(
    style_kwds={"fillColor": "blue", "color": "black", "weight": 1, "fillOpacity": 0.2},
    tiles="CartoDB positron",
    column="name",
    legend=True,
    legend_kwds={"caption": "Region"}
)

Then we can inspect the exposure objects (buildings and roads) of the Delft-FIAT model, by loading the model with the HydroMT-FIAT plugin.

In [ ]:
# Get the path of the FIAT model
fn_fiat = STATIC_DATA_DIR  / "fiat"
# Read the FIAT model using HydroMT-FIAT
fiat = FiatModel(root=str(fn_fiat), mode="r")
fiat.read()
# Get the geodataframe with exposure data
gdf = fiat.exposure.get_full_gdf(fiat.exposure.exposure_db)
# Plot the region and the secondary_object_types of the exposure data
gdf.explore(column="secondary_object_type", 
                name="Exposure types",
                tiles="CartoDB positron"
                )

## 📄 **Step 2**: Build a **basic** FloodAdapt Database

In this step we will create a basic FloodAdapt database, using only the mandatory configuration parameters.  

### 📚 **Step 2a**: Build the configuration from the **Database Builder classes**

To create the configuration object we can use the `ConfigModel` class of the Database Builder.

The mandatory configuration attributes include the `name` of the database and the `database_path` which points to the location where the database will be stored. Then, a `unit_system` needs to be specified, which can be either `metric` or `imperial`. The `unit_system` will determine the default units used in the database. For the output visualizations, scaling values need to be specified for each output type, using the `gui` attribute. 

Last, the overland SFINCS model and the Delft-FIAT model need to be specified. The SFINCS model is specified using the `sfincs_overland` attribute, which includes the path to the SFINCS model, defined by the attribute `name` and the vertical reference that the model has, defined by `reference`. The Delft-FIAT model is specified using the `fiat` attribute, which points to the path of the Delft-FIAT model.

In [ ]:
database_path=(STATIC_DATA_DIR / "Database").as_posix()  # Where the database will be stored
unit_system=db.UnitSystems.imperial # Define the unit system for the database
gui=db.GuiConfigModel(
    max_flood_depth=5,
    max_aggr_dmg=1e6,
    max_footprint_dmg=250000,
    max_benefits=5e6,
)  # Define the max values for each type of layer in the GUI
sfincs_overland=FloodModel(
    name=(STATIC_DATA_DIR / "overland").as_posix(),
    reference="MSL"  # This is the vertical reference for the SFINCS model
)  # Define the overland SFINCS model path and vertical reference
fiat=(STATIC_DATA_DIR / "fiat").as_posix()  # Define the FIAT model path

config_model = db.ConfigModel(
    name="charleston_example_basic", # unique name for the database
    database_path=database_path,
    unit_system=unit_system,
    gui=gui,
    sfincs_overland=sfincs_overland,
    fiat=fiat
)

### 📖 **Step 2b**: Create a configuration file from a **dictionary**

An alternative approach, would be to create a dictionary with all the attributes and save it to a TOML file. Any path that is included in the file should be either an absolute path (using forward slashes `/`) or a relative path (using forward slashes `/` and relative to the path of the TOML file).

In [ ]:
# In this case we use relative paths, but absolute paths are also possible
config_dict = {
"name": "charleston_example_basic",
"database_path": "Database",
"sfincs_overland" : { 
    "name":"overland",
    "reference":"NAVD88"
    },
"fiat" : "fiat",
"unit_system" :"imperial",
"gui": {
"max_flood_depth": 5,
"max_aggr_dmg" : 1e6,
"max_footprint_dmg": 250000,
"max_benefits" : 5e6}
}

We can then save the configuration as a .toml file at the location of the other input data, that we defined in the dictionary as relative paths.

In [ ]:
# Save the configuration file
config_path = STATIC_DATA_DIR  / "db_config_basic.toml"
with open(config_path, "w") as f:
    toml.dump(config_dict, f)

The we can use the `read()` method of the `ConfigModel` class to read the configuration from the TOML file.

In [ ]:
config = db.ConfigModel.read(config_path)

We can then verify that the two created configurations are the same, by comparing the attributes of the two configuration objects.

In [ ]:
config == config_model

### 🏃‍♀️ **Step 2c**: Run the Database Builder

We can then run the Database Builder using the `DatabaseBuilder` class. The `DatabaseBuilder` class takes the configuration object we created previously as input. Then we can call the `build()` method to build the database. During building all the steps of the Database Builder are logged, so we can follow the progress of the building process and since a log file is saved, we can also check the log file after the building process is finished.

In [ ]:
# Run Option 2a -  DB-builder config from FloodAdapt classes
db_build = db.create_database(config_model, overwrite=True)

## 🗂️ **Step 3**. Build an **advanced** FloodAdapt Database

In the previous step we created a simple FloodAdapt database. Our simple database is limited in functionality, so in this next step we're adding more advanced configurations to expand its capabilities. 

For the advanced example we are only going to build the configuration from the Database Builder classes. Similarly to the simple example, the configuration can be created using a dictionary or toml file as well.

We are going to first construct each individual component of the configuration, and then we will combine them into a single configuration object.

### Probabilistic event set

We can add a probabilistic event set by providing the path to an existing event set with the attribute `probabilistic_set`. This will enable us to run risk and benefit scenarios in FloodAdapt (see [Risk and benefit analysis](../../../4_system_setup/index.qmd#Risk-and-benefit-analysis)). 

In case we provide a probabilistic event set to enable risk calculations, we can also specify the return periods that will be calculated from the event set in FloodAdapt during risk scenario runs. The default values are [1, 2, 5, 10, 25, 50, 100] years, but you can specify any other set of values with the `return_periods` attribute.

In [ ]:
probabilistic_set = str(Path(STATIC_DATA_DIR  / "test_set")) # Path to the prepared probabilistic set
return_periods = [1, 2, 5, 10, 25, 50, 100] # Here we just use the standard return periods

### SFINCS offshore model

If we have a SFINCS offshore model we can also pass this into the configuration with the `sfincs_offshore` attribute in the same way as the overland SFINCS model. This will allow us to run extra types of events (see [Simulating hurricane events and ‘ungauged’ historical events](../../../4_system_setup/index.qmd#Simulating-hurricane-events-and-'ungauged'-historical-events)). Let's first visualize the SFINCS offshore model to see its extent.

In [ ]:
# Use HydroMT-SFINCS to read the SFINCS model
off_sfincs_path = (STATIC_DATA_DIR / "offshore").as_posix()
sfincs = SfincsModel(root=off_sfincs_path, mode="r")
sfincs.read()
# Get the extent of the SFINCS model
gdf = sfincs.region[["geometry"]]
gdf["name"] = "offshore SFINCS Model Extent"
# Make a map of the SFINCS model extent
gdf.explore(
    style_kwds={"fillColor": "blue", "color": "black", "weight": 1, "fillOpacity": 0.2},
    tiles="CartoDB positron",
    column="name",
    legend=True,
    legend_kwds={"caption": "Region"}
)

Similarly, to the onshore SFINCS model, we can use a FloodModel class to define the path with the attribute `name` and its vertical reference with the attribute `reference` (which for an offshore models is typically 'MSL'). In case a correction is needed to bring MSL to present day conditions (see [Sea level offset for offshore simulations](../../../2_technical_docs/EventScenario.qmd#Sea-level-offset-for-offshore-simulations)), the `vertical_offset` attribute can be used to specify the correction.

In [ ]:
# Add the SFINCS offshore model
sfincs_offshore=FloodModel(
    name=off_sfincs_path,
    reference="MSL",
    vertical_offset=us.UnitfulLength(
        value=0.33, units=us.UnitTypesLength.feet # in this case we found from observations that there is an offset of 0.33 feet
    ))

### Historical hurricanes

If we have an offshore SFINCS model, we can run historical hurricanes as well if we are in a hurricane prone area. The configuration for running hurricanes or not, is set with the `cyclones` attribute, which in case we are in an area were hurricanes are not relevant we could turn to `False`. If this is set to `True` (which is the default value), the `cyclone_basin` attribute can be used to define the oceanic basin. The `Basins` class can be used to check the available basins. In the case of Charleston we are going to use `NA` - for North Atlantic. If this is not specified, all global basins will be downloaded.

In [ ]:
# Add cyclone tracks
cyclones=True
cyclone_basin=db.Basins.NA

### Tide gauge data

If there are water level observations from a close by tide gauge we can add them in the database, so they can directly be used during event creation (see [Downloading historical water levels](../../../4_system_setup/index.qmd#downloading-historical-water-levels)), by using the `tide_gauge` attribute. 

With the `source` attribute defined to `file`, and the use of the `file` attribute to define the path to a csv file with the tide gauge data, we can directly use the tide gauge data in the database. The vertical reference of the tide gauge data can be defined by the `ref` attribute. The CSV file should have two columns; the first contains a ‘datetime’ in the format DD/MM/YYYY HH:MM and the second column contains the water levels relative to the vertical reference defined.

In U.S., instead of manually providing a file, we can choose `db.TideGaugeSource.noaa_coops` as the `source` attribute, to find the closest tide gauge from the **NOOAA COOPS** tide gauge network. To avoid using a stations that is really far away, we can also specify a `max_distance` attribute, which will be used to filter the stations. If no station is found within the specified distance, the tide gauge data will not be added to the database. A set of water level references from this station will be added to the database as well. These include **"MLLW", "MHHW", "NAVD88", "MSL"**. The default reference of the observation is `MLLW`, which can be changed with the `ref` attribute.

In [ ]:
# Add tide gauges
tide_gauge=db.TideGaugeConfigModel(
    source=TideGaugeSource.noaa_coops,
    max_distance=us.UnitfulLength(
        value=100, units=us.UnitTypesLength.miles
    ),
)

### Observation points

By using the attribute `obs_points` we can add a list of observation points for which we will extract timeseries of water levels as an output of our event scenarios. We can add a list of `ObsPointModel` objects. Each of these objects must have a `name` and a `lat` and `lon` attribute. The `description` is optional.

In [ ]:
# Add observation points
obs_point=[
    ObsPointModel(
        name="Ashley_river",
        description="Ashley River - James Island Expy",
        lat=32.7765,
        lon=-79.9543,
    ),
    ObsPointModel(
        name='8665530',
        description="Charleston Cooper River Entrance",
        lat=32.78,
        lon=-79.9233,
    ),
]

### Sea Level Rise (SLR) scenarios

We can add sea level rise scenarios to be used in the projections of FloodAdapt, by using the `slr_scenarios` attribute, which should be a `SlrScenariosModel` object, with a `file` attribute pointing to a csv file with the columns:  **year, unit, scenario_1, scenario_2, ..., scenario_n**, and a `relative_to_year` attribute, which indicate the year relative to which these scenarios should be translated, when used in FloodAdapt.

Here we have created a slr scenario csv file like this already. Let's have a quick look in what the csv file looks like.

In [ ]:
slr_csv = (STATIC_DATA_DIR  / "slr.csv").as_posix()
pd.read_csv(slr_csv)

In [ ]:
# Add SLR scenarios
slr_scenarios=SlrScenariosModel(
    file=slr_csv,
    relative_to_year=2020,
)

### Social Vulnerability Index (SVI)

A social vulnerability (SVI) layer can be added to the database for extra infographics related to who is impacted. This can be done with the `svi` attribute which is a `db.SviConfigModel` object. The path to a geospatial file with the SVI layer is provided with the `file` attribute, the `field_name` attribute defines the column name within the spatial file with the SVI value and the `threshold` defines the threshold value for the SVI, which distinguishes between vulnerable and non-vulnerable areas. 

In our case we have already clipped an SVI layer (from https://www.atsdr.cdc.gov/place-health/php/svi/svi-data-documentation-download.html) to the Charleston area, so we can use it directly. Let's have a quick look in what the SVI layer looks like.

In [ ]:
svi_path = (STATIC_DATA_DIR / "CDC_svi_2020.gpkg").as_posix()
svi_layer = gpd.read_file(svi_path)
# Make a map of the SVI layer
svi_layer.explore(
    column="SVI",
    name="Social Vulnerability Index (SVI)",
    tiles="CartoDB positron",
    cmap="RdBu_r",
    scheme=None,
    style_kwds={"color": "black", "weight": 0.5, "fillOpacity": 0.7},
    legend=True,
    legend_kwds={"caption": "SVI (0.5=center)"},
    categorical=False,
    center=0.5
)

Now, let's create the SVI configuration object, using the `SviConfigModel` class.

In [ ]:
# Add social vulnerability index
svi=db.SviConfigModel(
    file=svi_path,
    field_name="SVI",
    threshold=0.5,
)

### Base Flood Elevation (BFE)

A base flood elevation (BFE) can be added to the database which allows users to elevate homes relative to this layer. This can be done with the `bfe` attribute which is a `db.SpatialJoinModel` object. The path to the geospatial vector file with the BFE layer is provided with the `file` attribute, the `field_name` attribute defines the column name within the spatial file with the BFE value.

In our case we have already created some dummy data, so we can use it directly. Let's have a quick look in what the BFE layer looks like.

In [ ]:
bfe_path = (STATIC_DATA_DIR / "bfe.geojson").as_posix()
bfe_layer = gpd.read_file(bfe_path)
# Make a map of the BFE layer
bfe_layer.explore(
    column="bfe",
    name="Base Flood Elevation (BFE)",
    tiles="CartoDB positron",
    cmap="Blues",
    scheme=None,
    style_kwds={"color": "black", "weight": 0.5, "fillOpacity": 0.7},
    legend=True,
    categorical=False,
)

Let's create the BFE configuration object, using the `SpatialJoinModel` class.

In [ ]:
# Add base flood elevation
bfe=db.SpatialJoinModel(
    file=str(Path(STATIC_DATA_DIR / "bfe.geojson")),
    name="bfe",
    field_name="bfe",
)

### Aggregation areas

- `aggregation_areas`: A list of dictionaries with the entries: name, file path, field_name. Aggregates the exposure into larger spatial groups o summarize impacts on larger scale. 

In [ ]:
# Add aggregation areas
aggregation_areas=[
db.SpatialJoinModel(
   name="aggr_lvl_1",
   file=(STATIC_DATA_DIR / "aggr_lvl_1.geojson").as_posix(),
   field_name="name",
),
db.SpatialJoinModel(
   name="aggr_lvl_2",
   file=(STATIC_DATA_DIR /"aggr_lvl_2.geojson").as_posix(),
   field_name="name",
),
]

To refine our Delft-FIAT model we can add additional inputs or update default settings. 

Sometimes our Delf-FIAT model exposure is in point data By adding the `building_footprints`attribute we can download data from Open Street Map (OSM) using the `FootprintsOptions.OSM` object. e use the building footprints to create the visualizations for the impacts on the building footprint level.  
The `fiat_buildings_name` and `fiat_roads_name` are set to *"buildings"* and *"roads"*, respectively as default. These names capture the geometry names of these assets in the Delft-FIAT model. If you specified different names for these geometry, you must change them here. By defining a value in the `road_width` attribute we can capture the realistic width of the road. 

In [ ]:
# Add additional Delft-FIAT parameters
building_footprints=db.FootprintsOptions.OSM
fiat_buildings_name="buildings"
fiat_roads_name="roads"
road_width=5

Now, that we created all the individual objects we can compile them in the `ConfigModel`. 

In [ ]:
# Compile ConfigModel
config_model = db.ConfigModel(
        name = "charleston_example_advanced",
        database_path=(STATIC_DATA_DIR / "Database").as_posix(),
        unit_system=db.UnitSystems.imperial,  
        gui=db.GuiConfigModel(
                max_flood_depth=5,
                max_aggr_dmg=1e6,
                max_footprint_dmg=250000,
                max_benefits=5e6,
        ), 
        sfincs_overland=FloodModel(
                name=(STATIC_DATA_DIR / "overland").as_posix(),
                reference="NAVD88", 
        ),
        fiat=(STATIC_DATA_DIR / "fiat").as_posix(), 
        probabilistic_set=probabilistic_set,
        return_periods=return_periods,
        sfincs_offshore=sfincs_offshore,
        slr_scenarios=slr_scenarios,
        tide_gauge=tide_gauge,
        cyclones=cyclones,
        cyclone_basin=cyclone_basin,
        obs_point=obs_point,
        # aggregation_areas=aggregation_areas,
        building_footprints=db.FootprintsOptions.OSM,
        fiat_buildings_name=fiat_buildings_name,
        fiat_roads_name=fiat_roads_name,
        bfe=bfe,
        svi=svi,
        road_width=road_width,
)

## 🏃‍♀️ **Step 4**. Run the Database Builder

We are ready to run the Database Builder with the configuration that we just created above. First we are going to run **Option 3a** - in which we generated the configuration using the FloodAdapt classes.

In [ ]:
# Run Option 2a -  DB-builder config from FloodAdapt classes
db_build = db.create_database(config_model)

## 🚀 **Step 5.** - Reading-in the FloodAdapt database
Now that we built the database we can open it and continue to work with it.  

In the other example notebooks in this repository you can find the instructions on how to create and save the single components to create a full scenario (events, measures, strategies, projections) in your database.

In [ ]:
settings = Settings(
    DATABASE_ROOT=Path(STATIC_DATA_DIR / "Database").resolve(),
    DATABASE_NAME="charleston_example_advanced"
)
fa = FloodAdapt(database_path=settings.database_path)

## **Finished!**

Congratulations! You created your own FloodAdapt database and know now how to initiate it. 